# Homework 8 - Andrea Lazzari

### Neurosience - Excitatory-Inhibitory Two Neurons System

	
Study the stability of the excitatory-inhibitory two neurons system for the parameters:

$$

M = \begin{pmatrix}

M_{EE} & M_{EI} \\
M_{IE} & M_{II}

\end{pmatrix} \quad \quad

h = \begin{pmatrix}
h_E \\
h_I
\end{pmatrix}

$$

$$

M = \begin{pmatrix}

1.25 & -1 \\
1 & 0

\end{pmatrix} \quad \quad

h = \begin{pmatrix}
10 \\
-10
\end{pmatrix}

$$

The time constants are $\tau_E = 10$ ms and different $\tau_I$. 
  
Simulate a trajectory in the stable regime and one in the limit cycle regime.

---

## The Framework

The dynamics of the excitatory-inhibitory two neurons system is described by:

$$
\begin{cases}

\tau_E \cdot \dfrac{dv_E}{dt} = -v_E + \psi(h_E + M_{EE}v_E - M_{EI}v_I ) \\
\\
\tau_I \cdot \dfrac{dv_I}{dt} = -v_I + \psi(h_I + M_{IE}v_E - M_{II}v_I ) \\


\end{cases}
$$

where:
* $v_E$ and $v_I$ are the membrane potentials of the excitatory and inhibitory neurons, respectively
* $\tau_E$ and $\tau_I$ are the time constants of the excitatory and inhibitory neurons
* $h_E$ and $h_I$ are the external inputs to the excitatory and inhibitory neurons
* $M_{EE}$, $M_{EI}$, $M_{IE}$, $M_{II}$ are the synaptic weights between the excitatory and inhibitory neurons, concerning mutual and self interactions

The function $\psi$ we consider the ReLU function:

$$
\psi(x) = \begin{cases}
x & \text{if } x > 0 \\
0 & \text{otherwise}

\end{cases}
$$

---

### Stability Analysis

As a first step we find the equilibrium points of the system:

$$
\begin{cases}
\dfrac{dv_E}{dt} = 0 \\
\\
\dfrac{dv_I}{dt} = 0
\end{cases}

\quad \to \quad \text{fixed point} \quad (v_E^*, v_I^*) = \left( \dfrac{M_{EI}h_E - M_{EE}h_I}{M_{EE}M_{II} - M_{EI}M_{IE}}, \dfrac{M_{IE}h_E - M_{II}h_I}{M_{EE}M_{II} - M_{EI}M_{IE}} \right)
$$


The Jacobian matrix of the system in the fixed point is:

$$

J \rvert_{v^*} = \begin{pmatrix}
-\dfrac{(1 - M_{EE})}{\tau_E} & \dfrac{M_{EI}}{\tau_E} \\
\\
\dfrac{M_{IE}}{\tau_I} & -\dfrac{(1 - M_{II})}{\tau_I}

\end{pmatrix}


$$

The eigenvalues of the Jacobian matrix are:

$$
\lambda = \dfrac{1}{2} \cdot \bigg [ \dfrac{M_{EE} - 1}{\tau_E} + \dfrac{M_{II} - 1}{\tau_I} \pm 
\sqrt{ \bigg( \dfrac{M_{EE}}{\tau_E} - \dfrac{M_{II} - 1}{\tau_E} \bigg)^2 + \dfrac{4 \cdot M_{EI} \cdot M_{IE}}{\tau_E \cdot \tau_I}  }  \bigg]
$$

With the parameters given, the argument of the square root is negative, and the two eighenvalues form a complex conjugate pair.
This means that the behaviour around the fixed point is oscillatory, and the trajectory either spirals into the fixed point,  if the real part of the eigenvalues is negative, or out from the fixed point, if the real part is positive. 
In the first case, the fixed point is stable, while in the second case it is unstable.

However, because of the non-linear term involved into equations, the unstable trajectory does not expand indefinitely, but it converges toward a so-called **limit cycle**. The onset of these different behaviours, either stationary or sustained oscillations, is ultimately controlled by the parameter
$\tau_I$ that we are free to control.   

The point that separates these two regimes is known as a **bifurcation point**.

---

### Simulation

In [17]:
import numpy as np
import scipy as scp

import matplotlib.pyplot as plt
import matplotlib

from scipy.integrate import odeint

import pandas as pd 
import plotly.express as px

from dash import Dash, dcc, html
import plotly.graph_objs as go
from dash.dependencies import Input, Output , State
from jupyter_dash import JupyterDash



In [20]:
#excitatory-inhibitory two neurons system simulation

M_EE = 1.25         
M_EI = -1      
M_IE = 1          
M_II = 0

h_E = 10
h_I = -10

tau_E = 10
tau_I = 7   # parameter to change

v_E_0 = 19
v_I_0 = 19

def ReLU(x):
	return max(0.0, x)

def system(s, t, M_EE, M_EI, M_IE, M_II, h_E, h_I, tau_E, tau_I):
    v_E, v_I = s
       
    dv_E_dt = (1/tau_E) * (-v_E + ReLU(h_E + M_EE * v_E + M_EI * v_I ) )               
    dv_I_dt = (1/tau_I) * (-v_I + ReLU(h_I + M_IE * v_E + M_II * v_I ) )              
    
    return [dv_E_dt, dv_I_dt]

t = np.linspace(0, 1000, 1567)   # time_domain

s0 = [v_E_0, v_I_0]   # initial conditions


To plot the dynamics we make an interactive plot:

* The user can choose the time constant $\tau_I$ of the inhibitory neuron
* Then with Submit button the user can plot the dynamics of the system

In [21]:
pd.options.plotting.backend = "plotly"

app = JupyterDash(__name__)

app.layout = html.Div([
        dcc.Slider(min=0, max=50, step=1, value=tau_I, id ='slider', tooltip={"placement": "bottom", "always_visible": True}),
        html.Button(id='submit-button', type='submit', children='Submit', style={'padding':10,'fontSize':18}),
        html.Div(id='slider-output')], 
                      )

@app.callback(Output('slider-output', 'children'),
             [Input('submit-button', 'n_clicks')],
             [State('slider', 'value')])
                                     
                                     
                                     
def update_output(clicks, slider):
        if clicks is not None:
        
            tau_I = slider
            
            solution = odeint(system, s0, t, args=(M_EE, M_EI, M_IE, M_II, h_E, h_I, tau_E, tau_I))
            
            arr = np.array([t, solution[:,0], solution[:,1]]).T
            df = pd.DataFrame(arr, columns = ['t[s]', 'V_Exc', 'V_Inh'])

            fig = px.line(df , x='t[s]', title='Excitatory-Inhibitory Two Neurons System - tau_I = ' + str(tau_I) + ' ms',
                          y=['V_Exc', 'V_Inh'], 
              labels={'value':'Dynamics', 'variable':'Neuron', 't[s]':'t [s]'},
              markers=True, height=600, width=1100)

            
            fig.update_layout(font=dict(size=20))
            
            return dcc.Graph(figure=fig)
            

app.run_server(mode='inline')

In particular:

* When $\tau_I < 40$ ms, the real part of the eigenvalues is **negative**, and the fixed point is **stable**.


* When $\tau_I \geq 40$ ms, the real part of the eigenvalues is **positive**, and the fixed point is **unstable**.
